Tóm tắt thông tin mô hình:
- Mô hình sử dụng GXBoost
- Kiểm định whole dataset cross-validation: Để tận dụng toàn bộ tập dataset để train mô hình (vì dataset hiện chỉ giới hạn  4k dòng)


## TD forecast on python

In [207]:
%load_ext autoreload
%autoreload 2
import pyodbc
import pandas as pd
import numpy as np

import xgboost as xgb
# from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
# from sklearn.metrics import r2_score, mean_squared_error
# from sklearn.linear_model import LinearRegression
# from sklearn import model_selection

import pickle

server = '10.16.157.42'
database = 'RB_DATA'

connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
sql_query = '''
    SELECT *
    FROM USER_DATA.HIENNPD3.FCST_EOP_TD_MAINMODEL 
    WHERE PERIOD LIKE '2023-%' OR PERIOD LIKE  '2024-%'
    ORDER BY PERIOD DESC
'''
cursor.execute(sql_query)  
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)

cursor.close()
conn.close() 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [208]:
df['PERIOD'] = pd.to_datetime(df['PERIOD']) 
df_v2 = df 
df_v2 = df_v2[['PERIOD', 'CURRENCY_2', 'TERM_'
               , 'EOP_CM', 'EOP_LM'
               ,'UMASS_MIN','UMASS_MAX','UMASS_VPB','RANKING_UMASS_VPB'
               , 'INTERBANK_SHORT', 'INTERBANK_LONG', 'DISCOUNTRATE'
               , 'REFINANCINGRATE', 'CIE_VOL', 'ADB_VOL']]
df_v2['YEAR'] = df_v2['PERIOD'].dt.year 
df_v2['MONTH'] = df_v2['PERIOD'].dt.month 
df_v2['NO_ACT_DATE'] = df_v2['PERIOD'].dt.daysinmonth

C:\Users\hiennpd3\AppData\Local\Temp\ipykernel_27340\3472777732.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v2['YEAR'] = df_v2['PERIOD'].dt.year
C:\Users\hiennpd3\AppData\Local\Temp\ipykernel_27340\3472777732.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_v2['MONTH'] = df_v2['PERIOD'].dt.month
C:\Users\hiennpd3\AppData\Local\Temp\ipykernel_27340\3472777732.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [209]:
# df_v2
df_v2[df_v2['PERIOD']== '2023-12-01']

,PERIOD,CURRENCY_2,TERM_,EOP_CM,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,YEAR,MONTH,NO_ACT_DATE
40,2023-12-01,VND,From 18M,1916.356266,1898.013270,3.9,5.50,4.8,11,5.08,6.68,3,4.500000,-14.027270,1908.714485,2023,12,31
41,2023-12-01,NON-VND,less than 1M,11.810962,189.962519,2.6,4.35,3.6,5,0.17,0.44,3,4.500000,0.000000,14.664382,2023,12,31
42,2023-12-01,NON-VND,12-18M,223.575497,220.054959,3.9,5.50,4.8,11,5.08,6.68,3,4.500000,-0.015756,227.022648,2023,12,31
43,2023-12-01,VND,1-3M,40444.404859,20338.878685,2.6,4.35,3.6,5,1.33,4.54,3,4.500000,-126.702250,38811.416628,2023,12,31
44,2023-12-01,VND,6-9M,133466.842511,114831.295974,3.9,5.50,4.8,7,5.08,6.68,3,4.500000,-718.561196,131654.762967,2023,12,31
45,2023-12-01,NON-VND,1-3M,271.064790,239.807927,2.6,4.35,3.6,5,1.33,4.54,3,4.500000,-0.007739,277.758043,2023,12,31
46,2023-12-01,NON-VND,From 18M,55.865429,56.097673,3.9,5.50,4.8,11,5.08,6.68,3,4.500000,0.000000,61.712944,2023,12,31
47,2023-12-01,VND,4-5M,459.850287,387.146713,2.6,4.35,3.6,5,1.33,4.54,3,4.500000,-1.638594,478.706360,2023,12,31
48,2023-12-01,VND,less than 1M,559.483602,33181.211331,2.6,4.35,3.6,5,0.17,0.44,3,4.500000,-0.132338,361.085474,2023,12,31
49,2023-12-01,NON-VND,6-9M,172.470744,167.580676,3.9,5.50,4.8,7,5.08,6.68,3,4.500000,-0.024472,183.073226,2023,12,31


In [210]:
from datetime import datetime, timedelta

def get_eop_lm(year, month, term, currency, dataset_name):
    previous_month = datetime(year, month, 1) - timedelta(days=1)
    previous_year = previous_month.year
    previous_month = previous_month.month

    eop_lm = dataset_name.loc[(dataset_name['YEAR'] == previous_year)
                              & (dataset_name['MONTH'] == previous_month)
                              & (dataset_name['TERM_'] == term)
                              & (dataset_name['CURRENCY_2'] == currency), 'EOP_CM'].values

    if len(eop_lm) > 0:
        return eop_lm[0]
    else:
        return np.nan

### 1. Forecast EOP_CM -> Term

In [211]:
df_v3 = df_v2.groupby(['YEAR', 'MONTH', 'NO_ACT_DATE', 'CURRENCY_2', 'TERM_']).agg({'EOP_CM': 'sum'
                                                            , 'EOP_LM': 'sum'                                                                         
                                                            , 'UMASS_MIN': 'max'
                                                            , 'UMASS_MAX': 'max'
                                                            , 'UMASS_VPB': 'max'
                                                            , 'RANKING_UMASS_VPB': 'max'

                                                            , 'INTERBANK_SHORT': 'max'
                                                            , 'INTERBANK_LONG': 'max'
                                                            , 'DISCOUNTRATE': 'max'
                                                            , 'REFINANCINGRATE': 'max'
                                                            
                                                            , 'CIE_VOL': 'sum'
                                                            , 'ADB_VOL': 'sum'}).reset_index()
float_columns = ['EOP_CM','EOP_LM', 'UMASS_MIN', 'UMASS_MAX', 'UMASS_VPB', 'RANKING_UMASS_VPB'
                 , 'INTERBANK_SHORT' , 'INTERBANK_LONG', 'DISCOUNTRATE', 'REFINANCINGRATE' , 'CIE_VOL']

df_v3[float_columns] = df_v3[float_columns].astype(float)


In [212]:
# Tạo trường 'EOP_LM' trong DataFrame 'df_2'
# df_v3['EOP_LM_Fc'] = df_v3.apply(lambda row: get_eop_lm(row['YEAR']
#                                                      , row['MONTH']
#                                                      , row['TERM_']
#                                                      , row['CURRENCY_2'], df_v3), axis=1)


# Tạo trường 'EOP_LM' trong DataFrame 'df_v3'
df_v3['EOP_LM_FUNC'] = df_v3.apply(lambda row: row['EOP_LM'] if ((row['YEAR'] == 2023) and (row['MONTH'] == 1)) else get_eop_lm(row['YEAR'],
                                                                                                                                row['MONTH'],
                                                                                                                                row['TERM_'],
                                                                                                                                row['CURRENCY_2'], df_v3),
                                                                                                                                axis=1)
df_v3['CIE_Rate'] = np.where( (df_v3['CIE_VOL'] == 0) | (df_v3['CIE_VOL'] is None) | (df_v3['ADB_VOL'] == 0) |  (df_v3['ADB_VOL'] is None) , 
                             0,
                             (df_v3['CIE_VOL'] / (-df_v3['ADB_VOL'])) * (365/ df_v3['NO_ACT_DATE'] ) * 100 
                             )

In [213]:
mapping_TERM_ = {'less than 1M': 0,
                '1-3M': 1,
                '4-5M': 2,
                '6-9M': 3,
                '10-11M': 4,
                '12-18M': 5,
                'From 18M': 6
                }
mapping_CURRENCY_2 = {'VND': 0, 'NON-VND': 1}

In [214]:
categorical_cols = [ 'CURRENCY_2', 'TERM_']

def encode_engine(df_name, column_name):
    if column_name == 'CURRENCY_2':
        map_rule = mapping_CURRENCY_2
    elif column_name == 'TERM_':
        map_rule = mapping_TERM_
    df_name[column_name] = df_name[column_name].map(map_rule)
    return df_name

for x in categorical_cols:
    df_v4 = encode_engine(df_v3, x)

In [215]:
# df_v4.to_csv(R'C:\Users\hiennpd3\OneDrive - VPBank\AA Team\2. TD forecast model\3. Architecture\Package\SQL & Manual input raw\NGHIENCUU.CSV')
df_v4[(df_v4['YEAR']== 2024)&(df_v4['MONTH']== 3)]

,YEAR,MONTH,NO_ACT_DATE,CURRENCY_2,TERM_,EOP_CM,EOP_LM,UMASS_MIN,UMASS_MAX,UMASS_VPB,RANKING_UMASS_VPB,INTERBANK_SHORT,INTERBANK_LONG,DISCOUNTRATE,REFINANCINGRATE,CIE_VOL,ADB_VOL,EOP_LM_FUNC,CIE_Rate
182,2024,3,31,1,1,0.0,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,311.190877,0.0
183,2024,3,31,1,4,0.0,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,NaN,0.0
184,2024,3,31,1,5,0.0,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,212.802887,0.0
185,2024,3,31,1,2,0.0,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,2.939174,0.0
186,2024,3,31,1,3,0.0,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,179.575271,0.0
187,2024,3,31,1,6,0.0,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,48.022415,0.0
188,2024,3,31,1,0,0.0,0.0,1.73,5.1,2.5,5.0,1.47,1.75,3.0,4.5,0.0,0.0,13.178794,0.0
189,2024,3,31,0,1,0.0,0.0,1.73,2.9,2.5,5.0,1.88,2.65,3.0,4.5,0.0,0.0,48751.958546,0.0
190,2024,3,31,0,4,0.0,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,379.812864,0.0
191,2024,3,31,0,5,0.0,0.0,2.99,4.7,4.2,2.0,4.51,4.07,3.0,4.5,0.0,0.0,54153.532686,0.0


In [218]:
from sklearn.model_selection import GridSearchCV

df_train = df_v4[~(df_v4['YEAR'] == 2024)]
x_train = df_train.drop(['EOP_CM'], axis=1)
y_train = df_train['EOP_CM']

df_test = df_v4[(df_v4['YEAR'] == 2024) & df_v4['MONTH'] != 3]
x_test = df_test.drop(['EOP_CM'], axis=1)
y_test = df_test['EOP_CM']

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0]
}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=3)
grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_

xgb_model = xgb.XGBRegressor(**best_params)
xgb_model.fit(x_train, y_train)

best_model = grid_search.best_estimator_
# Lưu mô hình vào tệp
filename = 'best_model.pickle'
with open(filename, 'wb') as file:
    pickle.dump(best_model, file)

y_train_pred = xgb_model.predict(x_train)
y_test_pre =  xgb_model.predict(x_test)
df_train['FCST_EOP_CM'] = y_train_pred
df_test['FCST_EOP_CM'] = y_test_pre
score_train = xgb_model.score(x_train, y_train)
score_test = xgb_model.score(x_test, y_test)

score_test, score_train


C:\Users\hiennpd3\AppData\Local\Temp\ipykernel_27340\529840346.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['FCST_EOP_CM'] = y_train_pred


(0.9908124047480278, 0.999973888230612)

#### Giả định input CIE được truyền vào
Def function để thực hiện predict trong quá trình người nhập dữ liệu

In [217]:
INPUT_CIE_U1M = 0.01
INPUT_CIE_1M3M = 0.01
INPUT_CIE_4M5M = 0.01
INPUT_CIE_6M9M = 0.01
INPUT_CIE_10M11M = 0.01
INPUT_CIE_12M18M = 0.01
INPUT_CIE_OV18M = 0.01
INPUT_CIE_SPECIAL = 0.01


### 2. Forecast NET -> movement_status

### 3. Combine các giá trị ở các năm vào một bảng
- Combine
- Xuất giá trị vào csv
